In [5]:
using Revise

In [6]:
using ForwardDiff

In [7]:
using UnPack

In [8]:
using LinearAlgebra
using ImmersedLayers

In [9]:
using HeatForecast

In [35]:
function temperature1(x,obs::TemperatureObservations,gridConfig::constructGrids)
	@unpack config, sens = obs
	@unpack Nq, state_id = config
	@unpack g, cache = gridConfig
	Ny = length(sens)
	x_ids = state_id["heater x"]
  	y_ids = state_id["heater y"]
  	q_ids = state_id["heater q"]
  	r_ids = state_id["heater r"]
	
	xq = x[x_ids]
	yq = x[y_ids]
	qq = x[q_ids]
	rq = x[r_ids]

	T = zeros_grid(cache)
	xg, yg = coordinates(T,g)
	Temp = zeros(Ny)

	@inbounds for k in 1:Nq
		T = zeros_grid(cache)
		@inbounds for (j,yy) in enumerate(yg), (i,xx) in enumerate(xg)
    		if ((xx-xq[k])^2+(yy-yq[k])^2) < rq[k]^2
        		T[i,j] = -qq[k]
    		end
		end
		inverse_laplacian!(T,cache)
		Tfield = interpolatable_field(T,g)
		Temp .+= [Tfield(real(sens[j]), imag(sens[j])) for j in 1:Ny]

	end
	return Temp
end

temperature1 (generic function with 1 method)

In [29]:
@unpack cache, g = gridConfig
T = zeros_grid(cache)
@which inverse_laplacian!(T,cache)

inverse_laplacian!(w::GridData, cache::BasicILMCache) in ImmersedLayers at C:\Users\Hanieh\.julia\packages\ImmersedLayers\6MOTR\src\grid_operators.jl:153

In [11]:
function jacobia(x,obs::TemperatureObservations,gridConfig::constructGrids)
    @unpack sens, config = obs
    @unpack Nq, state_id = config
    Nx = state_length(state_id)
    Ny = size(sens,1)
    J = zeros(Ny,Nx)
    J = ForwardDiff.jacobian(y->temperature1(y,obs,gridConfig),x)
    return J
end

jacobia (generic function with 1 method)

### True data

In [12]:
zq = [0.5+1.2im]
qq = [1.0]
rq = [0.5]
Nq_true = length(zq);

### Setting up the observer for the true data

In [13]:
config_true, x_true = get_config_and_state(zq,qq,rq)

ϵmeas_data = [1e-5,1e-4,2.5e-4,3.75e-4,5e-4,6.25e-4,8e-4,1e-3,1.5e-3,2e-3,3e-3,4e-3];
x_true

4-element Vector{Float64}:
 0.5
 1.2
 1.0
 0.5

### Heater estimation

In [14]:
Nq_estimator = 1
δ = 0.01
config_estimator = HeaterConfig(Nq_estimator)


# ranges to confine the prior mean to
xr = (-2.0,2.0)
yr = (0.01,2) #(0.01,4) #(0.01,2.0)
qr = (-2,2)
rr = (0,2)
bounds = create_state_bounds(xr,yr,qr,rr,config_estimator);
bounds

4-element Vector{Tuple{Float64, Float64}}:
 (-2.0, 2.0)
 (0.01, 2.0)
 (-2.0, 2.0)
 (0.0, 2.0)

In [19]:
# bounds for numerical computations
xr = (-2.0,2.0)
yr = (-2,2) #(0.01,4) #(0.01,2.0)
qr = (-2,2)
rr = (0,2)
bounds_num = create_state_bounds(xr,yr,qr,rr,config_estimator);
bounds_num

4-element Vector{Tuple{Float64, Float64}}:
 (-2.0, 2.0)
 (-2.0, 2.0)
 (-2.0, 2.0)
 (0.0, 2.0)

### Getting results for a specific case

In [20]:
Δx = 0.01
gridConfig = constructGrids(Δx,bounds_num)

constructGrids{Tuple{Float64, Float64}, BasicILMCache{0, GridScaling, Nodes{Primal, 406, 406, Float64, Matrix{Float64}}, 2, BodyList, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Regularize{0, false}, RegularizationMatrix{Edges{Primal, 406, 406, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Edges{Primal, 406, 406, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, RegularizationMatrix{Nodes{Primal, 406, 406, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Nodes{Primal, 406, 406, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, Nothing, Nothing, Nothing, Nothing, Laplacian{406, 406, Float64, true, false}, Edges{Primal, 406, 406, Float64, Vector{Float64}}, Nodes{Dual, 406, 406, Float64, Matrix{Float64}}, Nothing, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Nothing}}(0.01, (-2.0, 2.0), (-2.0, 2.

In [21]:
Nx = length(x_true)
Nsens = 3
ϵmeas = 5e-4
obs_true, ystar, H, Σϵ, Σx = get_truth_data(Nsens,ϵmeas,x_true,config_true,gridConfig)

(TemperatureObservations{4, 3, Vector{ComplexF64}, HeaterConfig{Dict{Any, Any}}}(ComplexF64[-1.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im], HeaterConfig{Dict{Any, Any}}(1, Dict{Any, Any}("heater q" => [3], "heater r" => [4], "heater y" => [2], "heater x" => [1]), 0.0)), [-0.6552327514805953, -0.6065663400781611, -0.6065663400781612], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [2.5e-7 0.0 0.0; 0.0 2.5e-7 0.0; 0.0 0.0 2.5e-7], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0])

In [22]:
ystar

3-element Vector{Float64}:
 -0.6552327514805953
 -0.6065663400781611
 -0.6065663400781612

In [41]:
J = ForwardDiff.jacobian(y->temperature1(y,obs_true,gridConfig),x_true)

MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{var"#37#38", Float64}, Float64, 4})
Closest candidates are:
  (::Type{T})(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:200
  (::Type{T})(::T) where T<:Number at boot.jl:772
  (::Type{T})(!Matched::AbstractChar) where T<:Union{AbstractChar, Number} at char.jl:50
  ...

In [18]:
J = zeros(Nsens,Nx)
jacobia(x_true,obs_true,gridConfig)

MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{var"#11#12"{TemperatureObservations{4, 3, Vector{ComplexF64}, HeaterConfig{Dict{Any, Any}}}, constructGrids{Tuple{Float64, Float64}, BasicILMCache{0, GridScaling, Nodes{Primal, 406, 209, Float64, Matrix{Float64}}, 2, BodyList, VectorData{0, Float64, Vector{Float64}}, ScalarData{0, Float64, Vector{Float64}}, Regularize{0, false}, RegularizationMatrix{Edges{Primal, 406, 209, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Edges{Primal, 406, 209, Float64, Vector{Float64}}, VectorData{0, Float64, Vector{Float64}}}, RegularizationMatrix{Nodes{Primal, 406, 209, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, InterpolationMatrix{Nodes{Primal, 406, 209, Float64, Matrix{Float64}}, ScalarData{0, Float64, Vector{Float64}}}, Nothing, Nothing, Nothing, Nothing, Laplacian{406, 209, Float64, true, false}, Edges{Primal, 406, 209, Float64, Vector{Float64}